### Cчитывание всех файлов с генами
И подсчет количества законченных экспериментов
(Берется количество генов в папке)

In [14]:
import glob
import os

path = os.getcwd()

print("Директория: " + path)

txtfiles_genes = []
for file in glob.glob("SimulationData/Genes/*.txt"):
    txtfiles_genes.append(file)
    
print("Количество завершенных экспериментов: " + str(len(txtfiles_genes)))

Директория: C:\Users\Public\Documents\Unity Projects\CellularAutomations\Statistics
Количество завершенных экспериментов: 620
